# Main

# Path + imports

In [352]:
from pathlib import Path

PROJECT_ROOT = Path.cwd()
RAW = PROJECT_ROOT / "data" / "raw"
PROCESSED = PROJECT_ROOT / "data" / "processed"

PROCESSED.mkdir(parents=True, exist_ok=True)

In [353]:
import pandas as pd

REVIEWS_PATH = RAW / "olist_order_reviews_dataset.csv"
ORDERS_PATH = RAW / "olist_orders_dataset.csv"
PAYMENTS_PATH = RAW / "olist_order_payments_dataset.csv"

reviews_raw = pd.read_csv(REVIEWS_PATH)
orders_raw = pd.read_csv(ORDERS_PATH)
payments_raw = pd.read_csv(PAYMENTS_PATH)

print("reviews_raw:", reviews_raw.shape)
print("orders_raw:", orders_raw.shape)
print("payments_raw:", payments_raw.shape)

reviews_raw.head()

reviews_raw: (99224, 7)
orders_raw: (99441, 8)
payments_raw: (103886, 5)


,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [354]:
# Helpers
import re


def parse_time(df: pd.DataFrame, cols):
    df = df.copy()
    for c in cols:
        df[c] = pd.to_datetime(df[c], errors="coerce")
    return df


def basic_clean_text(s: str) -> str:
    s = "" if pd.isna(s) else str(s)
    s = s.lower()
    s = re.sub(r"http\S+|www\.\S+", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s


def resolve_payment_type(series: pd.Series):
    types = series.dropna().unique()
    if len(types) == 1:
        return types[0]
    if len(types) == 0:
        return pd.NA
    return "mixed"

# Reviews dataset

### Initial review and date parse time

In [355]:
# Initial
reviews = reviews_raw.copy()
# Parse datetime cols
reviews = parse_time(reviews, ["review_answer_timestamp", "review_creation_date"])
reviews[["review_answer_timestamp", "review_creation_date"]].info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 2 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   review_answer_timestamp  99224 non-null  datetime64[ns]
 1   review_creation_date     99224 non-null  datetime64[ns]
dtypes: datetime64[ns](2)
memory usage: 1.5 MB


### Text

In [356]:
# Text flags (has_text, length)
reviews["has_text"] = (
    reviews["review_comment_message"].fillna("").astype(str).str.strip().ne("")
)
reviews["text_length"] = (
    reviews["review_comment_message"].fillna("").astype(str).str.len()
)
reviews["text_clean"] = reviews["review_comment_message"].apply(basic_clean_text)

# print("Has text:", reviews["has_text"])
print("Text length:", reviews["text_length"], "\n")
print("Text cleaned:", reviews["text_clean"].head())


Text length: 0          0
1          0
2          0
3         37
4        100
        ... 
99219      0
99220      0
99221     67
99222      0
99223     85
Name: text_length, Length: 99224, dtype: int64 

Text cleaned: 0                                                     
1                                                     
2                                                     
3                recebi bem antes do prazo estipulado.
4    parabéns lojas lannister adorei comprar pela i...
Name: text_clean, dtype: object


### Response time on days:

In [357]:
# Parse time on days (float)
reviews["response_time_days"] = (
    reviews["review_answer_timestamp"] - reviews["review_creation_date"]
).dt.total_seconds() / (60 * 60 * 24)

reviews["response_time_days"]

0        0.907627
1        1.128623
2        1.608611
3        0.918125
4        1.435336
           ...   
99219    7.721181
99220    2.837986
99221    1.382442
99222    1.541123
99223    0.876262
Name: response_time_days, Length: 99224, dtype: float64

### Binary target: bad reviews (<=2) and good reviews (>2)

In [358]:
reviews["bad_review"] = (reviews["review_score"] <= 2).astype(int)
reviews["bad_review"].value_counts()

bad_review
0    84649
1    14575
Name: count, dtype: int64

In [359]:
reviews_ready = reviews[
    [
        "review_id",
        "order_id",
        "review_score",
        "bad_review",
        "review_creation_date",
        "review_answer_timestamp",
        "response_time_days",
        "has_text",
        "text_length",
        "review_comment_message",
        "text_clean",
    ]
]
print(reviews_ready.shape)
reviews_ready.head(2)

(99224, 11)


,review_id,order_id,review_score,bad_review,review_creation_date,review_answer_timestamp,response_time_days,has_text,text_length,review_comment_message,text_clean
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,0,2018-01-18,2018-01-18 21:46:59,0.907627,False,0,NaN,
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,0,2018-03-10,2018-03-11 03:05:13,1.128623,False,0,NaN,


### Sanity checks


In [360]:
assert reviews_ready["review_id"].notna().all()
assert reviews_ready["order_id"].notna().all()

assert reviews_ready["review_score"].between(1, 5).all()
print(reviews_ready["review_score"])

0        4
1        5
2        5
3        5
4        5
        ..
99219    5
99220    5
99221    5
99222    4
99223    1
Name: review_score, Length: 99224, dtype: int64


In [361]:
# Grain: order id in reviews df
duplicated_review_id = reviews_ready["review_id"].duplicated().mean() * 100
duplicated_order_id = reviews_ready["order_id"].duplicated().mean() * 100
print("Duplicate rate review_id:", duplicated_review_id)
print("Duplicate rate order_id:", duplicated_order_id)

Duplicate rate review_id: 0.8203660404740789
Duplicate rate order_id: 0.555309199387245


In [362]:
# Response time negative
review_response_time = (reviews_ready["response_time_days"] < 0).sum()
print("Negative response time:", review_response_time)

Negative response time: 0


### Output (parquet)

# MVP2


## Orders dataset

In [363]:
orders = orders_raw.copy()
orders = parse_time(
    orders, ["order_delivered_customer_date", "order_estimated_delivery_date"]
)

orders_subset = orders[
    [
        "order_id",
        "order_status",
        "order_purchase_timestamp",
        "order_delivered_customer_date",
        "order_estimated_delivery_date",
    ]
].copy()

orders_subset["delivery_delay_days"] = (
    orders_subset["order_delivered_customer_date"]
    - orders_subset["order_estimated_delivery_date"]
).dt.total_seconds() / 86400

orders_subset.head()


,order_id,order_status,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,delivery_delay_days
0,e481f51cbdc54678b7cc49136f2d6af7,delivered,2017-10-02 10:56:33,2017-10-10 21:25:13,2017-10-18,-7.107488
1,53cdb2fc8bc7dce0b6741e2150273451,delivered,2018-07-24 20:41:37,2018-08-07 15:27:45,2018-08-13,-5.355729
2,47770eb9100c2d0c44946d9cf07ec65d,delivered,2018-08-08 08:38:49,2018-08-17 18:06:29,2018-09-04,-17.245498
3,949d5b44dbf5de918fe9c16f97b45f8a,delivered,2017-11-18 19:28:06,2017-12-02 00:28:42,2017-12-15,-12.980069
4,ad21c59c0840e6cb83a9ceb5573f8159,delivered,2018-02-13 21:18:39,2018-02-16 18:17:02,2018-02-26,-9.238171


## Payment dataset

In [364]:
payments = payments_raw.copy()

payments_order = (
    payments.groupby("order_id")
    .agg(
        payment_installments=("payment_installments", "max"),
        payment_value=("payment_value", "sum"),
        payment_type=("payment_type", resolve_payment_type),
    )
    .reset_index()
)

payments_order.head(2)


,order_id,payment_installments,payment_value,payment_type
0,00010242fe8c5a6d1ba2dd792cb16214,2,72.19,credit_card
1,00018f77f2f0320c557190d7a144bdd3,3,259.83,credit_card


## `reviews_enriched`: Merge order and payment datasets

In [365]:
reviews_enriched = reviews_ready.merge(orders_subset, how="left", on="order_id").merge(
    payments_order, how="left", on="order_id"
)
print(reviews_enriched.shape)
reviews_enriched.head()

(99224, 19)


,review_id,order_id,review_score,bad_review,review_creation_date,review_answer_timestamp,response_time_days,has_text,text_length,review_comment_message,text_clean,order_status,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,delivery_delay_days,payment_installments,payment_value,payment_type
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,0,2018-01-18,2018-01-18 21:46:59,0.907627,False,0,NaN,,delivered,2018-01-11 15:30:49,2018-01-17 18:42:41,2018-02-02,-15.220359,8.0,397.26,credit_card
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,0,2018-03-10,2018-03-11 03:05:13,1.128623,False,0,NaN,,delivered,2018-02-28 12:25:19,2018-03-09 23:17:20,2018-03-14,-4.029630,1.0,88.09,credit_card
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,0,2018-02-17,2018-02-18 14:36:24,1.608611,False,0,NaN,,delivered,2018-02-03 09:56:22,2018-02-16 17:28:48,2018-03-09,-20.271667,1.0,194.12,credit_card
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,0,2017-04-21,2017-04-21 22:02:06,0.918125,True,37,Recebi bem antes do prazo estipulado.,recebi bem antes do prazo estipulado.,delivered,2017-04-09 17:41:13,2017-04-20 09:08:35,2017-05-10,-19.619039,1.0,222.84,credit_card
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,0,2018-03-01,2018-03-02 10:26:53,1.435336,True,100,Parabéns lojas lannister adorei comprar pela I...,parabéns lojas lannister adorei comprar pela i...,delivered,2018-02-10 10:59:03,2018-02-28 16:33:35,2018-03-09,-8.310012,10.0,1333.25,credit_card


### Reviews text: dataset for NLP (exclude rows don't contain text )

In [366]:
reviews_enriched["response_time_negative"] = (
    reviews_enriched["response_time_days"] < 0
).astype(int)
reviews_enriched.loc[
    reviews_enriched["response_time_days"] < 0, "response_time_days"
] = pd.NA

reviews_enriched["is_delivered"] = (
    reviews_enriched["order_status"] == "delivered"
).astype(int)
reviews_enriched["has_delivery_delay"] = (
    reviews_enriched["delivery_delay_days"].notna().astype(int)
)

print(reviews_enriched["response_time_negative"].value_counts())
print(reviews_enriched["is_delivered"].value_counts())
reviews_enriched["has_delivery_delay"].value_counts()


response_time_negative
0    99224
Name: count, dtype: int64
is_delivered
1    96361
0     2863
Name: count, dtype: int64


has_delivery_delay
1    96359
0     2865
Name: count, dtype: int64

In [367]:
# Grain review_id
assert reviews_enriched.shape[0] == reviews_ready.shape[0]

print(reviews_enriched["review_id"].is_unique)
print((reviews_enriched["order_id"].duplicated().mean() * 100))

# Check isna on columns
print(
    reviews_enriched[["delivery_delay_days", "payment_type", "payment_value"]]
    .isna()
    .mean()
    .sort_values(ascending=False)
    .head(10)
)


False
0.555309199387245
delivery_delay_days    0.028874
payment_type           0.000010
payment_value          0.000010
dtype: float64


In [368]:
duplicate_rate = reviews_enriched["review_id"].duplicated().mean() * 100
duplicate_n = reviews_enriched["review_id"].duplicated().sum()
print(duplicate_n, duplicate_rate)


814 0.8203660404740789


### Deduplicate review_id (keeping rows with text, then last creation date)

In [ ]:
tmp = reviews_enriched.copy()
print("previous delete rows duplicated:", tmp.shape)

tmp["has_usable_text"] = tmp["text_clean"].fillna("").str.len().gt(0)

tmp = tmp.sort_values(["review_id", "has_usable_text", "review_creation_date"])
print(tmp["review_id"].duplicated().sum())

reviews_enriched_clean = (
    tmp.drop_duplicates("review_id", keep="last")
    .drop(columns=["has_usable_text"])
    .copy()
)

print("after delete rows duplicated:", reviews_enriched_clean.shape)


previous delete rows duplicated: (99224, 22)
814
after delete rows duplicated: (98410, 22)


In [370]:
out1 = PROCESSED / "reviews_enriched.parquet"
reviews_enriched_clean.to_parquet(out1, index=False, engine="pyarrow")

### NLP DataFrame. The reason of this dataset is review order level for nlp purposes 

In [371]:
nlp_df = reviews_enriched_clean.copy()
print("nlp dataframe shape:", nlp_df.shape)
nlp_df.columns

nlp dataframe shape: (98410, 22)


Index(['review_id', 'order_id', 'review_score', 'bad_review',
       'review_creation_date', 'review_answer_timestamp', 'response_time_days',
       'has_text', 'text_length', 'review_comment_message', 'text_clean',
       'order_status', 'order_purchase_timestamp',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'delivery_delay_days', 'payment_installments', 'payment_value',
       'payment_type', 'response_time_negative', 'is_delivered',
       'has_delivery_delay'],
      dtype='object')

### Sanity check: nlp dataset

In [ ]:
# Sanity checks
assert nlp_df["review_id"].is_unique
assert nlp_df["text_clean"].notna().all()
assert nlp_df["bad_review"].isin([0, 1]).all(), "It's not binary"

### Keep the rows that have text

In [373]:
print("previous delete rows:", nlp_df.shape)

nlp_df = nlp_df[nlp_df["has_text"]].copy()
nlp_df = nlp_df[nlp_df["text_clean"].str.len() > 0].copy()

print("after deleted rows without text:", nlp_df.shape)
nlp_df.head()

previous delete rows: (98410, 22)
after deleted rows without text: (40641, 22)


,review_id,order_id,review_score,bad_review,review_creation_date,review_answer_timestamp,response_time_days,has_text,text_length,review_comment_message,...,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,delivery_delay_days,payment_installments,payment_value,payment_type,response_time_negative,is_delivered,has_delivery_delay
50129,00020c7512a52e92212f12d3e37513c0,e28abf2eb2f1fbcbdc2dd0cd9a561671,5,0,2018-04-25,2018-04-26 14:55:36,1.621944,True,69,A entrega foi super rápida e o pendente é lind...,...,2018-04-17 21:54:15,2018-04-24 18:12:22,2018-05-10,-15.241412,2.0,167.96,credit_card,0,1,1
58174,00046a69550325aea5fb89f65c7387f2,9fbda7367628952bc36c3512c46d887b,5,0,2018-07-01,2018-07-02 12:32:24,1.522500,True,52,"GOSTEI DA CAPINHA DE CELULAR, VEIO COMO EU ESP...",...,2018-06-22 13:33:54,2018-06-30 15:28:42,2018-07-31,-30.355069,1.0,25.22,credit_card,0,1,1
92591,0005534973388c830bb858cfba83b17b,a589caa6892ceacc6bbf2f8cc30a8ad4,5,0,2017-09-01,2017-09-02 13:04:08,1.544537,True,55,otimo produto. prazo cumprido. sabor tambem mu...,...,2017-08-15 17:32:48,2017-08-31 23:34:54,2017-09-21,-20.017431,2.0,140.20,credit_card,0,1,1
98276,00055e36e9608fe969231e551983a69c,f5fea26ab547eec920e6f8ecdc5c37e4,5,0,2017-11-25,2017-11-27 11:37:22,2.484282,True,64,O produto foi entregue muito antes do esperado...,...,2017-11-20 19:38:16,2017-11-24 19:56:40,2017-12-07,-12.168981,1.0,187.34,boleto,0,1,1
69053,0005949d4c047d64863a6874338139ba,640883769bc5da4a077725646dc9995a,5,0,2017-06-20,2017-06-22 23:19:41,2.972002,True,53,"Bom eu já sabia que era,mas é muito bonito.rec...",...,2017-06-05 18:20:04,2017-06-19 13:18:09,2017-06-30,-10.445729,8.0,126.43,credit_card,0,1,1


### Output

In [374]:
out2 = PROCESSED / "reviews_nlp.parquet"
nlp_df.to_parquet(out2, index=False, engine="pyarrow")
print("Saved:", out2)


Saved: c:\Users\Jesus\Documents\olist_nlp\data\processed\reviews_nlp.parquet
